In [1]:
import zipfile
from pulp import LpProblem
import os
import gzip
from gurobipy import read, GRB

import graph_nets as gn
import sonnet as snt
import tensorflow as tf
from graph_nets import graphs
from graph_nets import utils_tf

import numpy as np

from pyscipopt import Model, quicksum, SCIP_RESULT, SCIP_PARAMSETTING,SCIP_EVENTTYPE,Eventhdlr

In [4]:
#read the path to a problem and get the answer of the
def collect_every_solution(path):
    model = Model()
    model.readProblem(path)

    # Disable presolving and enable solution collection
    model.setPresolve(SCIP_PARAMSETTING.OFF)
    #model.setParam('limits/solutions', -1)  # Collect all solutions



    class SolutionCollector(Eventhdlr):
        def __init__(self):
            super().__init__()
            self.model = model
            self.solutions = []

        def eventinit(self):
            self.model.catchEvent(SCIP_EVENTTYPE.BESTSOLFOUND, self)

        def eventexit(self):
            self.model.dropEvent(SCIP_EVENTTYPE.BESTSOLFOUND, self)

        def eventexec(self, event):
            #sol = self.model.getBestSol()
            #x_val = self.model.getVars()

           # self.solutions.append(x_val)
            
            sol = self.model.getBestSol()
            sol_values = {var.name: self.model.getSolVal(sol, var) for var in self.model.getVars()}
            self.solutions.append(sol_values)


    # Create an instance of the event handler
    solution_collector = SolutionCollector()

    # Include the event handler in the model
    model.includeEventhdlr(solution_collector, "SolutionCollector", "Collects all feasible solutions")

    # Solve the model
    model.optimize()


    sol_set = solution_collector.solutions

    #现在我们拥有了路径中问题的所有可行解的集合，可以用它来构建二部图了


    return sol_set

In [ ]:
problem_file_path = 'data/extract/bm_ex'
solution_file_path = 'data/extract/bm_ex_solution'

def get_solution_from_file(path):
    #make sure the solution file exists
    if not os.path.exists(solution_file_path):
        os.makedirs(solution_file_path)
    
    for filename in os.listdir(path):
        print(filename)
        file_path = os.path.join(path, filename)
        file_ans=collect_every_solution(file_path)
        
        #新建文件的答案文档
        solution_file = os.path.join(solution_file_path, filename)
        # 打开文件进行写入
        with open(solution_file, 'w') as f:
            for item in file_ans:
                # 将数组中的每个元素转换为字符串并写入文件
                # 假设数组中的每个元素都是字符串或者可以被转换为字符串
                f.write(str(item) + '\n')  
        break
        
        
get_solution_from_file(problem_file_path)